<h1><center>Canceled bookings at a hotel</center></h1>


You have been assigned the task of building a model that will predict whether or not a customer of a hotel will cancel their booking. The data for this assingment is found in the csv file `hotel_clf`

<br> 
<div>
<img src="https://5.imimg.com/data5/PC/BL/MY-33192851/hotel-reservation-services-500x500.jpg" width="400"/>
</div>
<br> 
If the model predicts that a customer will cancel their booking, that customer will be sent a special deal to try to keep the customer from cancel the booking. If the prediction is correct (a True Positive), the expected gain is 1000 SEK. However, if the prediction is wrong (a False Positive), the expected loss is 500 SEK. 

Your goal is to build the most profitable model possible.

<hr style="border:1px solid pink"> </hr>

## Q1 | Choose Metric

Reason about which metric you think will be best to optimize your model for.

- Recall?
- Precision?
- Accuracy?
- F1-score?

Make a decision about which metric you think will lead to the most profitable model

In [ ]:
# Answer:

## Q2 | Data prepatation

- Prepare your data so that you end up with a clean and preprocessed train and test set
    
    
- Instructions for train test split:    
    - Test size = 0.2
    - Random state = 42

## Q3 | Build a LogReg Model

Guidelines:
- Use a LogisticRegression model
    - Random state = 42
- Use the metric you decided on in the previous question

- You are not allowed to change the model after looking at the performance on test data
- Your models predictions on test data will be translated into SEK. I.e:
    - 10 TP = 10 * 1 000 SEK = +10 000 
    - 10 FP = 10 * -500 SEK = -5 000 SEK
        - Expected Value from model = +5 000 SEK 
        
        
After you have trained your model, make predictions for your test data and calculate the profitable of the model

## Q4 | Build a RandomForestClassifier model

- Use a RandomForestClassifier model:
    - random_state = 42


- After you have trained your model, make predictions for your test data and calculate the profitable of the model

- Which model was more profitable, the LogReg or the RandomForestClassifier?

## Q5 | Did you choose the right metric? 

Calculate the profitablity for the RandomForestClassifier for all 4 different metrics. Then rank order the outcome. I.e.:

- RFC (precision) = 1
- RFC (accuracy) = 2
- ...
- ...


***Note:*** You don't have to use a param_grid for this question, just run the RandomForest with default settings